In [1]:
import mne
import os 
import glob
import re
import gc 
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from joblib import Parallel, delayed
from eeg_auto_tools.transforms import (Sequence, FilterBandpass, SetMontage, 
        BadChannelsDetector, Rereference, AutoICA, BridgeInterpolate,
        Interpolate, ChannelSelector, Raw2Epoch)
from mne_bids import read_raw_bids, find_matching_paths
from tqdm_joblib import tqdm_joblib

gc.collect()

mne.utils.set_config('MNE_USE_CUDA', 'false')
qc_dataframe_file = "DatasetInfo.csv"
os.makedirs("ICA", exist_ok=True)

/home/aysurkov/.conda/envs/ml_cb/lib/python3.10/site-packages/tqdm_joblib/__init__.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# Preprocessing
pip1 = Sequence(
    ch_selector = ChannelSelector(exclude=['ECG', 'RESP1', 'EOG', 'M1', 'M2', 'BIP', 'Cz', 'E127', 'E126', 'E48', 'E119', 'E17']),
    montager = SetMontage('GSN-HydroCel-128'), #waveguard64
)

pipeline_QC = Sequence(
    ffilter = FilterBandpass(l_freq=0.1, h_freq=45, notch_freq=50),
    detector = BadChannelsDetector(method_noise="auto", method_bridge="ed"),
)
    
def preprocessing(file, qc_dataframe_file):
    path_obj = Path(file)
    new_path_obj = Path("ICA") / f"{path_obj.stem}"
    ica_dir =  str(new_path_obj)
    os.makedirs(new_path_obj, exist_ok=True)
    if os.path.exists(os.path.join(ica_dir, "ica_solution.fif")) and os.path.exists(os.path.join(ica_dir, "ICLabel_info.pkl")):
        return
    
    pipeline_prep = Sequence(
        ffilter = FilterBandpass(l_freq=1, h_freq=100, notch_freq=50, report=False),
        rerefer = Rereference(exclude='bads', report=False),
        #brinter = BridgeInterpolate(reset_bridges=False),
        #inter = Interpolate(),
        ica = AutoICA(n_components=0.99999, output_dir=ica_dir, exclude_bads=True, report=True),
    )
    
    #raw = mne.io.read_raw(file, preload=True, verbose=False)
    raw = read_raw_bids(file, verbose=False)

    pre_raw = pip1(raw, cash=False)
    del raw

    pipeline_QC(pre_raw, cash=False)

    data_dict = {**pipeline_QC.detector.repo_data}
    #pre_raw.info['bads'] = raw_qc.info["bads"]
    pre_raw.info['bads'] = list(set(data_dict['HighAmp']) | set(data_dict['LowAmp']) | set(data_dict['Bridged']))
    print(f"HighAmp:{set(data_dict['HighAmp'])} LowAmp:{set(data_dict['LowAmp'])} Bridged:{set(data_dict['Bridged'])} Noise_Rate:{set(data_dict['Noise_Rate'])}")

    picks = mne.pick_types(pre_raw.info, eeg=True, exclude='bads')

    # Если пусто — пропускаем всё
    if picks.size == 0:
        print("⚠️ Нет доступных EEG‑каналов - файл будет проигнорирован")
        return
    data = pre_raw.get_data(picks=picks)
    if np.isnan(data).any() or np.isinf(data).any():
        print(f"⚠️ Файл {file} содержит NaN/Inf — пропускаем обработку")
        return

    try:
        pipeline_prep(pre_raw, cash=False)
    except Exception as e:
        print(f"⚠️ Ошибка обработки файла {file}: {e}")
        return
        
    del pre_raw
    data_dict = {**pipeline_QC.detector.repo_data, **pipeline_prep.ica.repo_data}
    data_dict["file"] = os.path.basename(file)
    data_dict['N_bad_channels'] = len(set(data_dict['HighAmp']) | set(data_dict['LowAmp']) | set(data_dict['Bridged']))
    df = pd.DataFrame([data_dict])
    first_columns = ["file", 'N_bad_channels', "HighAmp", "LowAmp", "Bridged", "Noise_Rate", 
                    "FINAL", "exclude_idx", "exclude_labels", "Clusters"]
    cols = first_columns + [col for col in df.columns if not (col in first_columns)]
    df = df[cols]

    df.to_csv(qc_dataframe_file, mode='a', sep=';', index=False, header= not os.path.isfile(qc_dataframe_file), encoding='utf-8-sig')
    gc.collect()

def process_file(file):
    preprocessing(file, qc_dataframe_file)


# Список папок с датасетами
datasets = ["ds005505", "ds005506", "ds005507"]
bids_root_base = "DATA"

# Собираем все пути
bids_paths = []
for ds in datasets:
    root = f"{bids_root_base}/{ds}"
    bids_paths.extend(
        find_matching_paths(
            root=root,
            suffixes="eeg",
            extensions=".set",
            ignore_json=True
        )
    )


In [ ]:
# Параллельная обработка с прогресс-баром
with tqdm_joblib(tqdm(total=len(bids_paths))) as progress:
    Parallel(n_jobs=10, timeout=None)(delayed(process_file)(bp) for bp in bids_paths)

  0%|          | 0/4559 [00:00<?, ?it/s]

  0%|          | 0/4559 [00:00<?, ?it/s]

/tmp/job-2691887/ipykernel_223591/511155425.py:29: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
/tmp/job-2691887/ipykernel_223591/511155425.py:29: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
/tmp/job-2691887/ipykernel_223591/511155425.py:29: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
/tmp/job-2691887/ipykernel_223591/511155425.py:29: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
/tmp/job-2691887/ipykernel_223591/511155425.py:29: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
/tmp/job-2691887/ipykernel_223591/511155425.py:29: RuntimeWarning: Data will be prelo

Figure(640x480)
Setting channel interpolation method to {'eeg': 'spline'}.


/home/aysurkov/.conda/envs/ml_cb/lib/python3.10/site-packages/eeg_auto_tools/quality_check.py:121: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  raw = mne.preprocessing.interpolate_bridged_electrodes(raw, bridged_idx, bad_limit=len(raw.ch_names))


...

/home/aysurkov/.conda/envs/ml_cb/lib/python3.10/site-packages/eeg_auto_tools/quality_check.py:121: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  raw = mne.preprocessing.interpolate_bridged_electrodes(raw, bridged_idx, bad_limit=len(raw.ch_names))
/tmp/job-2691887/ipykernel_223591/511155425.py:29: RuntimeWarning: Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file
/tmp/job-2691887/ipykernel_223591/511155425.py:29: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
/tmp/job-2691887/ipykernel_223591/511155425.py:29: RuntimeWarning: Unable to map the following column(s) to to MNE:
release_number: R2
ehq_total: 86.71
commercial_use: Yes
full_pheno: Yes
p_factor: -0.516
attention: 1.2009999999999998
internalizing: -0.555
externalizing: -0.6940000000000001
RestingState: available
DespicableMe: available
FunwithFracta

HighAmp:{'E88', 'E94'} LowAmp:{'E80', 'E106', 'E113', 'E67'} Bridged:{'E113', 'E67'} Noise_Rate:{'E31', 'E69', 'E68', 'E100', 'E124', 'E7', 'E12', 'E94', 'E115', 'E123', 'E118', 'E79', 'E80', 'E108', 'E39', 'E54', 'E2', 'E44', 'E76', 'E128', 'E106', 'E114', 'E55', 'E23', 'E56', 'E57', 'E16', 'E30', 'E95', 'E18', 'E105', 'E125', 'E49', 'E83', 'E4', 'E99', 'E10', 'E120', 'E89', 'E33', 'E88', 'E5', 'E13', 'E107'}
Writing ICA solution to /home/aysurkov/Vasya/ICA/sub-NDARBU607ZZ3_task-DiaryOfAWimpyKid_eeg/ica_solution.fif...
Figure(640x480)
Setting channel interpolation method to {'eeg': 'spline'}.


/home/aysurkov/.conda/envs/ml_cb/lib/python3.10/site-packages/eeg_auto_tools/quality_check.py:121: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  raw = mne.preprocessing.interpolate_bridged_electrodes(raw, bridged_idx, bad_limit=len(raw.ch_names))


Figure(640x480)
Setting channel interpolation method to {'eeg': 'spline'}.


/home/aysurkov/.conda/envs/ml_cb/lib/python3.10/site-packages/eeg_auto_tools/quality_check.py:121: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  raw = mne.preprocessing.interpolate_bridged_electrodes(raw, bridged_idx, bad_limit=len(raw.ch_names))
